In [288]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from pyHSICLasso import HSICLasso
import matplotlib.pyplot as plt
import scipy.io
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
import pandas as pd

In [289]:
# Load data and identify the classes
static_fc_data = scipy.io.loadmat('Static_functional_connectivity_ptsd_dc_filt.mat')
sfc_controls = static_fc_data['conn_sfc_dc_filt_controls']  # (125, 125, 56)
sfc_ptsd = static_fc_data['conn_sfc_dc_filt_ptsd']         # (125, 125, 34)
sfc_pcsptsd = static_fc_data['conn_sfc_dc_filt_pcsptsd']   # (125, 125, 84)

In [290]:
# Extract the upper triangular part of the matrices 
def extract_upper_triangle(connectivity_matrices):
    """Extract upper triangle (excluding diagonal) from connectivity matrices"""
    n_regions = connectivity_matrices.shape[0]
    n_subjects = connectivity_matrices.shape[2]
   
    # Get upper triangle indices
    upper_idx = np.triu_indices(n_regions, k=1)
   
    # Extract features for each subject
    features = np.zeros((n_subjects, len(upper_idx[0])))
    for i in range(n_subjects):
        features[i] = connectivity_matrices[upper_idx[0], upper_idx[1], i]
   
    return features

# Extract features
controls_features = extract_upper_triangle(sfc_controls)  # (56, 7750)
ptsd_features = extract_upper_triangle(sfc_ptsd)          # (34, 7750)
pcsptsd_features = extract_upper_triangle(sfc_pcsptsd)    # (84, 7750)

In [291]:
# Combine groups and create labels
X = np.vstack([controls_features, ptsd_features, pcsptsd_features])  # (174, 7750)
y = np.hstack([
    np.zeros(controls_features.shape[0]),      # 0 = controls
    np.ones(ptsd_features.shape[0]),           # 1 = PTSD & PTSD+PCS
    np.ones(pcsptsd_features.shape[0])       
])

In [292]:
# Split data into train and test sets (Without stratified sampling to replicate the reference paper)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42,
)

In [293]:
# Standardizing features after split to avoid potential data leakage and overfitting
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set: {X_train.shape}, Testing set: {X_test.shape}")

Training set: (139, 7750), Testing set: (35, 7750)


In [294]:
# Create feature names (node pairs)
n_regions = sfc_controls.shape[0]
upper_idx = np.triu_indices(n_regions, k=1)
feature_names = [f"Region_{i+1}_{j+1}" for i, j in zip(upper_idx[0], upper_idx[1])]

In [295]:
# HSIC Lasso for binary classification
# First convert labels to one-hot encoding
n_classes = len(np.unique(y))
y_train_onehot = np.zeros((len(y_train), n_classes))
for i in range(n_classes):
    y_train_onehot[:, i] = (y_train == i).astype(float)

In [296]:
# Number of top features to select per class
n_select = 200

# Initialize HSIC Lasso
hsic_lasso = HSICLasso()

# Store selected features for each class
all_selected_features = {}
selected_feature_indices = set()

for class_idx in range(n_classes):
    print(f"\nPerforming HSIC Lasso for Class {class_idx}")
    
    # Use the current class column as target
    y_class = y_train_onehot[:, class_idx]
    
    # Fit HSIC Lasso
    hsic_lasso.input(X_train_scaled, y_class)
    hsic_lasso.classification(num_feat=n_select, B=139)
    
    # Get selected feature indices
    selected = hsic_lasso.get_index()
    selected_feature_indices.update(selected)
    
    # Get feature scores
    scores = hsic_lasso.get_index_score()
    
    # Store selected features and their scores
    all_selected_features[f"Class_{class_idx}"] = {
        'features': [feature_names[idx] for idx in selected],
        'indices': selected,
        'scores': scores
    }
    
    print(f"Top {n_select} features for Class {class_idx}:")
    for i, idx in enumerate(selected):
        print(f"  {feature_names[idx]}: {scores[i]:.4f}")



Performing HSIC Lasso for Class 0
Block HSIC Lasso B = 139.
M set to 1.
Using Gaussian kernel for the features, Delta kernel for the outcomes.
Top 200 features for Class 0:
  Region_94_107: 0.1139
  Region_32_67: 0.1061
  Region_97_125: 0.0713
  Region_72_105: 0.0642
  Region_88_94: 0.0497
  Region_8_31: 0.0439
  Region_95_110: 0.0418
  Region_21_56: 0.0398
  Region_25_77: 0.0384
  Region_50_100: 0.0373
  Region_11_116: 0.0364
  Region_63_105: 0.0355
  Region_19_114: 0.0340
  Region_38_56: 0.0339
  Region_82_104: 0.0332
  Region_50_104: 0.0319
  Region_13_23: 0.0316
  Region_62_116: 0.0311
  Region_25_112: 0.0294
  Region_87_122: 0.0279
  Region_77_107: 0.0275
  Region_26_96: 0.0260
  Region_92_104: 0.0255
  Region_77_104: 0.0252
  Region_65_103: 0.0246
  Region_15_64: 0.0244
  Region_6_116: 0.0241
  Region_13_17: 0.0236
  Region_21_64: 0.0233
  Region_9_120: 0.0223
  Region_82_106: 0.0223
  Region_16_31: 0.0217
  Region_46_47: 0.0211
  Region_48_49: 0.0211
  Region_68_125: 0.0205
  R

In [297]:
def analyze_feature_distribution(all_selected_features, n_classes):
    # Create dictionary to store feature occurrences
    feature_occurrence = {}
    
    # Count occurrences of each feature across classes
    for class_idx in range(n_classes):
        features = all_selected_features[f"Class_{class_idx}"]['features']
        for feature in features:
            if feature not in feature_occurrence:
                feature_occurrence[feature] = []
            feature_occurrence[feature].append(class_idx)
    
    # Count features by occurrence 
    class_specific = {i: 0 for i in range(n_classes)}  # Features unique to each class
    class_pairs = {(i, j): 0 for i in range(n_classes) for j in range(i+1, n_classes)}  # Features shared by pairs
    #shared_by_all = 0  # Features shared by all classes
    
    for feat, classes in feature_occurrence.items():
        if len(classes) == 1:
            class_specific[classes[0]] += 1
        elif len(classes) == 2:
            class_pairs[tuple(sorted(classes))] += 1
    
    # Print numerical summary
    print("\nFeatures unique to each class:")
    for class_idx, count in class_specific.items():
        print(f"Class {class_idx}: {count}")
        
    print("\nFeatures shared between pairs:")
    for (c1, c2), count in class_pairs.items():
        print(f"Class {c1} and {c2}: {count}")
        
    print(f"Total unique features: {len(feature_occurrence)}")

# Run the analysis
feature_analysis = analyze_feature_distribution(all_selected_features, n_classes)


Features unique to each class:
Class 0: 0
Class 1: 0

Features shared between pairs:
Class 0 and 1: 131
Total unique features: 131


In [ ]:
# Get all unique selected features
all_features = []
for class_data in all_selected_features.values():
    all_features.extend(class_data['features'])
unique_features = list(set(all_features))

# Extract unique selected features
unique_selected_indices = list(selected_feature_indices)
X_train_selected = X_train_scaled[:, unique_selected_indices]
X_test_selected = X_test_scaled[:, unique_selected_indices]

# Split into train/validation/test
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train_selected, y_train, test_size=0.2, random_state=42
)

# Define models and parameter grids
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'SVM': SVC(probability=True, random_state=42),
    'MLP': MLPClassifier(max_iter=500, random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'XGBoost': XGBClassifier(eval_metric='logloss', random_state=42)
}

param_grids = {
    'Random Forest': {'n_estimators': [50, 100], 'max_depth': [None, 10]},
    'SVM': {'C': [0.1, 1], 'kernel': ['linear', 'rbf']},
    'MLP': {'hidden_layer_sizes': [(50,), (100,)], 'alpha': [0.0001, 0.001]},
    'Logistic Regression': {'C': [0.1, 1, 10]},
    'K-Nearest Neighbors': {'n_neighbors': [3, 5]},
    'Gradient Boosting': {'n_estimators': [50, 100], 'learning_rate': [0.01, 0.1]},
    'XGBoost': {'n_estimators': [50, 100], 'learning_rate': [0.01, 0.1]}
}

# Train and validate models
results = {}
for name, model in models.items():
    # Hyperparameter tuning
    grid_search = GridSearchCV(model, param_grids[name], cv=5, scoring='accuracy')
    grid_search.fit(X_train_final, y_train_final)
    best_model = grid_search.best_estimator_
    
    # Validation evaluation
    y_val_pred = best_model.predict(X_val)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    
    # Store results
    results[name] = {
        'model': best_model,
        'val_accuracy': val_accuracy,
        'best_params': grid_search.best_params_
    }

results_table = []

# Evaluate all models on the test set
for name in models:
    model = results[name]['model'].fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
    
    results_table.append({
        'Model': name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1
    })

results_df = pd.DataFrame(results_table)
print(results_df)

                 Model  Accuracy  Precision    Recall  F1-Score
0        Random Forest  0.800000   0.851852  0.800000  0.784459
1                  SVM  0.914286   0.925466  0.914286  0.912514
2                  MLP  0.885714   0.889111  0.885714  0.884354
3  Logistic Regression  0.914286   0.925466  0.914286  0.912514
4  K-Nearest Neighbors  0.885714   0.889111  0.885714  0.884354
5    Gradient Boosting  0.800000   0.851852  0.800000  0.784459
6              XGBoost  0.771429   0.836735  0.771429  0.748918
